# 서울 따릉이 대여량 예측

이 노트북에서는 서울시 공공자전거 '따릉이'의 대여량을 예측하는 머신러닝 모델을 구축합니다.

## 목차
1. 라이브러리 및 데이터 로드
2. 탐색적 데이터 분석 (EDA)
3. 데이터 전처리
4. 특성 공학 (Feature Engineering)
5. 모델링
6. 모델 평가 및 비교
7. 최종 예측 및 제출 파일 생성
8. **AutoGluon 모델링 (NEW)**
   - Medium Quality 모델
   - Best Quality 모델
   - 전체 모델 성능 비교

---

## 1. 라이브러리 및 데이터 로드

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 전처리 및 모델링
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import lightgbm as lgb

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'

print('라이브러리 로드 완료!')

In [ ]:
# 데이터 로드
DATA_PATH = 'data/'

train = pd.read_csv(DATA_PATH + 'train.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
submission = pd.read_csv(DATA_PATH + 'submission.csv')

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')
print(f'Submission shape: {submission.shape}')

In [ ]:
# 데이터 미리보기
train.head(10)

### 컬럼 설명

| 컬럼명 | 설명 | 단위 |
|--------|------|------|
| id | 고유 식별자 | - |
| hour | 시간 | 0~23시 |
| hour_bef_temperature | 1시간 전 기온 | °C |
| hour_bef_precipitation | 1시간 전 강수 여부 | 0(없음), 1(있음) |
| hour_bef_windspeed | 1시간 전 풍속 | m/s |
| hour_bef_humidity | 1시간 전 습도 | % |
| hour_bef_visibility | 1시간 전 가시거리 | m |
| hour_bef_ozone | 1시간 전 오존 농도 | ppm |
| hour_bef_pm10 | 1시간 전 미세먼지(PM10) | μg/m³ |
| hour_bef_pm2.5 | 1시간 전 초미세먼지(PM2.5) | μg/m³ |
| **count** | **따릉이 대여량 (타겟)** | 대 |

In [ ]:
# 데이터 정보 확인
print('=' * 50)
print('Train 데이터 정보')
print('=' * 50)
train.info()

In [ ]:
# 기초 통계량
train.describe()

---

## 2. 탐색적 데이터 분석 (EDA)

### 2.1 타겟 변수 분석

In [ ]:
# 타겟 변수 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 원본 분포
axes[0].hist(train['count'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('대여량 (count)')
axes[0].set_ylabel('빈도')
axes[0].set_title('따릉이 대여량 분포')
axes[0].axvline(train['count'].mean(), color='red', linestyle='--', label=f"평균: {train['count'].mean():.1f}")
axes[0].axvline(train['count'].median(), color='green', linestyle='--', label=f"중앙값: {train['count'].median():.1f}")
axes[0].legend()

# 로그 변환 분포
axes[1].hist(np.log1p(train['count']), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Log(대여량 + 1)')
axes[1].set_ylabel('빈도')
axes[1].set_title('따릉이 대여량 분포 (로그 변환)')

plt.tight_layout()
plt.show()

print(f"대여량 통계:")
print(train['count'].describe())

### 2.2 결측치 분석

In [ ]:
# 결측치 확인
print('=' * 50)
print('Train 데이터 결측치')
print('=' * 50)
missing_train = train.isnull().sum()
missing_train_pct = (missing_train / len(train) * 100).round(2)
missing_df = pd.DataFrame({'결측치 수': missing_train, '비율(%)': missing_train_pct})
print(missing_df[missing_df['결측치 수'] > 0])

print('\n' + '=' * 50)
print('Test 데이터 결측치')
print('=' * 50)
missing_test = test.isnull().sum()
missing_test_pct = (missing_test / len(test) * 100).round(2)
missing_df_test = pd.DataFrame({'결측치 수': missing_test, '비율(%)': missing_test_pct})
print(missing_df_test[missing_df_test['결측치 수'] > 0])

In [ ]:
# 결측치 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train 결측치
missing_train_plot = missing_train[missing_train > 0].sort_values(ascending=True)
if len(missing_train_plot) > 0:
    axes[0].barh(missing_train_plot.index, missing_train_plot.values, color='coral')
    axes[0].set_xlabel('결측치 수')
    axes[0].set_title('Train 데이터 결측치')
else:
    axes[0].text(0.5, 0.5, '결측치 없음', ha='center', va='center', fontsize=14)
    axes[0].set_title('Train 데이터 결측치')

# Test 결측치
missing_test_plot = missing_test[missing_test > 0].sort_values(ascending=True)
if len(missing_test_plot) > 0:
    axes[1].barh(missing_test_plot.index, missing_test_plot.values, color='steelblue')
    axes[1].set_xlabel('결측치 수')
    axes[1].set_title('Test 데이터 결측치')
else:
    axes[1].text(0.5, 0.5, '결측치 없음', ha='center', va='center', fontsize=14)
    axes[1].set_title('Test 데이터 결측치')

plt.tight_layout()
plt.show()

### 2.3 시간대별 대여량 분석

In [ ]:
# 시간대별 평균 대여량
hourly_avg = train.groupby('hour')['count'].mean()

plt.figure(figsize=(14, 5))
plt.bar(hourly_avg.index, hourly_avg.values, color='steelblue', edgecolor='black')
plt.xlabel('시간 (Hour)')
plt.ylabel('평균 대여량')
plt.title('시간대별 평균 따릉이 대여량')
plt.xticks(range(24))

# 출퇴근 시간 강조
plt.axvspan(7, 9, alpha=0.3, color='red', label='출근 시간')
plt.axvspan(17, 19, alpha=0.3, color='orange', label='퇴근 시간')
plt.legend()

plt.tight_layout()
plt.show()

print("시간대별 평균 대여량 (상위 5개):")
print(hourly_avg.sort_values(ascending=False).head())

### 2.4 기상 조건과 대여량 관계

In [ ]:
# 기온과 대여량 관계
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 기온 vs 대여량
axes[0, 0].scatter(train['hour_bef_temperature'], train['count'], alpha=0.3, s=10)
axes[0, 0].set_xlabel('기온 (°C)')
axes[0, 0].set_ylabel('대여량')
axes[0, 0].set_title('기온 vs 대여량')

# 습도 vs 대여량
axes[0, 1].scatter(train['hour_bef_humidity'], train['count'], alpha=0.3, s=10, color='green')
axes[0, 1].set_xlabel('습도 (%)')
axes[0, 1].set_ylabel('대여량')
axes[0, 1].set_title('습도 vs 대여량')

# 풍속 vs 대여량
axes[1, 0].scatter(train['hour_bef_windspeed'], train['count'], alpha=0.3, s=10, color='orange')
axes[1, 0].set_xlabel('풍속 (m/s)')
axes[1, 0].set_ylabel('대여량')
axes[1, 0].set_title('풍속 vs 대여량')

# 강수 여부별 대여량
rain_group = train.groupby('hour_bef_precipitation')['count'].mean()
axes[1, 1].bar(['비 안옴 (0)', '비 옴 (1)'], rain_group.values, color=['steelblue', 'coral'])
axes[1, 1].set_ylabel('평균 대여량')
axes[1, 1].set_title('강수 여부별 평균 대여량')

plt.tight_layout()
plt.show()

### 2.5 상관관계 분석

In [ ]:
# 상관관계 히트맵
corr_matrix = train.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('변수 간 상관관계 히트맵')
plt.tight_layout()
plt.show()

In [ ]:
# 타겟과의 상관관계
target_corr = corr_matrix['count'].drop('count').sort_values(ascending=False)

plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in target_corr.values]
plt.barh(target_corr.index, target_corr.values, color=colors)
plt.xlabel('상관계수')
plt.title('타겟(count)과의 상관관계')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

print("타겟(count)과의 상관관계:")
print(target_corr)

### 2.6 대기질과 대여량 관계

In [ ]:
# 대기질 관련 변수와 대여량
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 오존 vs 대여량
axes[0].scatter(train['hour_bef_ozone'], train['count'], alpha=0.3, s=10)
axes[0].set_xlabel('오존 농도 (ppm)')
axes[0].set_ylabel('대여량')
axes[0].set_title('오존 vs 대여량')

# PM10 vs 대여량
axes[1].scatter(train['hour_bef_pm10'], train['count'], alpha=0.3, s=10, color='brown')
axes[1].set_xlabel('미세먼지 PM10 (μg/m³)')
axes[1].set_ylabel('대여량')
axes[1].set_title('PM10 vs 대여량')

# PM2.5 vs 대여량
axes[2].scatter(train['hour_bef_pm2.5'], train['count'], alpha=0.3, s=10, color='purple')
axes[2].set_xlabel('초미세먼지 PM2.5 (μg/m³)')
axes[2].set_ylabel('대여량')
axes[2].set_title('PM2.5 vs 대여량')

plt.tight_layout()
plt.show()

---

## 3. 데이터 전처리

In [ ]:
# 데이터 복사
train_df = train.copy()
test_df = test.copy()

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

### 3.1 결측치 처리

In [ ]:
# 결측치가 있는 컬럼 확인
cols_with_missing = train_df.columns[train_df.isnull().any()].tolist()
print(f"결측치가 있는 컬럼: {cols_with_missing}")

# 수치형 컬럼의 결측치를 중앙값으로 대체
for col in train_df.columns:
    if train_df[col].isnull().sum() > 0:
        median_val = train_df[col].median()
        train_df[col].fillna(median_val, inplace=True)
        print(f"  {col}: 중앙값 {median_val:.2f}로 대체")

# Test 데이터도 동일하게 처리 (Train의 중앙값 사용)
for col in test_df.columns:
    if test_df[col].isnull().sum() > 0:
        # Train 데이터의 중앙값 사용
        if col in train.columns:
            median_val = train[col].median()
        else:
            median_val = test_df[col].median()
        test_df[col].fillna(median_val, inplace=True)

print("\n결측치 처리 완료!")
print(f"Train 결측치: {train_df.isnull().sum().sum()}")
print(f"Test 결측치: {test_df.isnull().sum().sum()}")

### 3.2 이상치 확인

In [ ]:
# 박스플롯으로 이상치 확인
numeric_cols = ['hour_bef_temperature', 'hour_bef_windspeed', 'hour_bef_humidity',
                'hour_bef_visibility', 'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    axes[i].boxplot(train_df[col].dropna())
    axes[i].set_title(col.replace('hour_bef_', ''))

# 마지막 subplot에 타겟 변수
axes[7].boxplot(train_df['count'])
axes[7].set_title('count (Target)')

plt.suptitle('변수별 박스플롯 (이상치 확인)', y=1.02)
plt.tight_layout()
plt.show()

---

## 4. 특성 공학 (Feature Engineering)

In [ ]:
def create_features(df):
    """
    새로운 특성을 생성하는 함수
    """
    df = df.copy()
    
    # 1. 시간대 구분 (출근/퇴근/주간/야간)
    def get_time_period(hour):
        if 7 <= hour <= 9:
            return 'morning_rush'    # 출근 시간
        elif 17 <= hour <= 19:
            return 'evening_rush'    # 퇴근 시간
        elif 10 <= hour <= 16:
            return 'daytime'         # 주간
        elif 20 <= hour <= 23:
            return 'evening'         # 저녁
        else:
            return 'night'           # 야간 (0-6시)
    
    df['time_period'] = df['hour'].apply(get_time_period)
    
    # 2. 러시아워 여부
    df['is_rush_hour'] = df['hour'].apply(lambda x: 1 if (7 <= x <= 9) or (17 <= x <= 19) else 0)
    
    # 3. 주간/야간 구분
    df['is_daytime'] = df['hour'].apply(lambda x: 1 if 6 <= x <= 20 else 0)
    
    # 4. 시간 사이클 특성 (시간의 주기성 반영)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # 5. 기온 구간 (쾌적한 온도 여부)
    df['is_comfortable_temp'] = df['hour_bef_temperature'].apply(
        lambda x: 1 if 15 <= x <= 25 else 0
    )
    
    # 6. 체감 불쾌 지수 (간단 버전)
    # 불쾌지수 = 0.81*기온 + 0.01*습도*(0.99*기온-14.3) + 46.3
    df['discomfort_index'] = (
        0.81 * df['hour_bef_temperature'] + 
        0.01 * df['hour_bef_humidity'] * (0.99 * df['hour_bef_temperature'] - 14.3) + 46.3
    )
    
    # 7. 미세먼지 등급
    def pm10_grade(pm10):
        if pm10 <= 30:
            return 1  # 좋음
        elif pm10 <= 80:
            return 2  # 보통
        elif pm10 <= 150:
            return 3  # 나쁨
        else:
            return 4  # 매우 나쁨
    
    df['pm10_grade'] = df['hour_bef_pm10'].apply(pm10_grade)
    
    # 8. 초미세먼지 등급
    def pm25_grade(pm25):
        if pm25 <= 15:
            return 1  # 좋음
        elif pm25 <= 35:
            return 2  # 보통
        elif pm25 <= 75:
            return 3  # 나쁨
        else:
            return 4  # 매우 나쁨
    
    df['pm25_grade'] = df['hour_bef_pm2.5'].apply(pm25_grade)
    
    # 9. 날씨 종합 점수 (자전거 타기 좋은 날씨)
    # 비 안오고, 적당한 온도, 낮은 미세먼지, 적당한 풍속
    df['good_weather_score'] = (
        (1 - df['hour_bef_precipitation']) * 2 +  # 비 안오면 +2
        df['is_comfortable_temp'] * 2 +           # 쾌적한 온도면 +2
        (df['pm10_grade'] <= 2).astype(int) +     # 미세먼지 보통 이하면 +1
        (df['hour_bef_windspeed'] <= 5).astype(int)  # 풍속 5m/s 이하면 +1
    )
    
    # 10. 가시거리 등급
    df['visibility_grade'] = pd.cut(
        df['hour_bef_visibility'],
        bins=[0, 200, 500, 1000, 2000, float('inf')],
        labels=[1, 2, 3, 4, 5]
    ).astype(int)
    
    return df

# 특성 생성
train_df = create_features(train_df)
test_df = create_features(test_df)

print(f"특성 생성 후 Train shape: {train_df.shape}")
print(f"특성 생성 후 Test shape: {test_df.shape}")

In [ ]:
# 새로운 특성 확인
new_features = ['time_period', 'is_rush_hour', 'is_daytime', 'hour_sin', 'hour_cos',
                'is_comfortable_temp', 'discomfort_index', 'pm10_grade', 'pm25_grade',
                'good_weather_score', 'visibility_grade']

print("새로 생성된 특성:")
train_df[new_features].head(10)

In [ ]:
# 범주형 특성 원-핫 인코딩
train_df = pd.get_dummies(train_df, columns=['time_period'], prefix='time')
test_df = pd.get_dummies(test_df, columns=['time_period'], prefix='time')

# Train에는 있지만 Test에 없는 컬럼 처리 (또는 그 반대)
train_cols = set(train_df.columns)
test_cols = set(test_df.columns)

# Test에 없는 컬럼 추가
for col in train_cols - test_cols:
    if col != 'count':  # 타겟 제외
        test_df[col] = 0

# Train에 없는 컬럼 추가
for col in test_cols - train_cols:
    train_df[col] = 0

print(f"원-핫 인코딩 후 Train shape: {train_df.shape}")
print(f"원-핫 인코딩 후 Test shape: {test_df.shape}")

### 4.1 새로운 특성과 타겟 간의 관계

In [ ]:
# 새로운 특성과 타겟 간의 관계 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 러시아워 여부별 대여량
rush_hour_avg = train_df.groupby('is_rush_hour')['count'].mean()
axes[0, 0].bar(['일반 시간', '러시아워'], rush_hour_avg.values, color=['steelblue', 'coral'])
axes[0, 0].set_ylabel('평균 대여량')
axes[0, 0].set_title('러시아워 여부별 평균 대여량')

# 쾌적한 온도 여부별 대여량
temp_avg = train_df.groupby('is_comfortable_temp')['count'].mean()
axes[0, 1].bar(['불쾌적 온도', '쾌적 온도'], temp_avg.values, color=['coral', 'green'])
axes[0, 1].set_ylabel('평균 대여량')
axes[0, 1].set_title('온도 쾌적성별 평균 대여량')

# 미세먼지 등급별 대여량
pm_avg = train_df.groupby('pm10_grade')['count'].mean()
axes[0, 2].bar(['좋음', '보통', '나쁨', '매우나쁨'], pm_avg.values, color='steelblue')
axes[0, 2].set_ylabel('평균 대여량')
axes[0, 2].set_title('미세먼지 등급별 평균 대여량')

# 날씨 종합 점수별 대여량
weather_avg = train_df.groupby('good_weather_score')['count'].mean()
axes[1, 0].bar(weather_avg.index, weather_avg.values, color='green')
axes[1, 0].set_xlabel('날씨 종합 점수')
axes[1, 0].set_ylabel('평균 대여량')
axes[1, 0].set_title('날씨 점수별 평균 대여량')

# 불쾌지수 vs 대여량
axes[1, 1].scatter(train_df['discomfort_index'], train_df['count'], alpha=0.3, s=10)
axes[1, 1].set_xlabel('불쾌지수')
axes[1, 1].set_ylabel('대여량')
axes[1, 1].set_title('불쾌지수 vs 대여량')

# 주간/야간별 대여량
daytime_avg = train_df.groupby('is_daytime')['count'].mean()
axes[1, 2].bar(['야간', '주간'], daytime_avg.values, color=['navy', 'orange'])
axes[1, 2].set_ylabel('평균 대여량')
axes[1, 2].set_title('주간/야간별 평균 대여량')

plt.tight_layout()
plt.show()

---

## 5. 모델링

In [ ]:
# 특성과 타겟 분리
target = 'count'
features = [col for col in train_df.columns if col not in ['id', 'count']]

X = train_df[features]
y = train_df[target]

X_test = test_df[features]
test_ids = test_df['id']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"\n사용할 특성 ({len(features)}개):")
print(features)

In [ ]:
# Train/Validation 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")

In [ ]:
# 평가 함수
def evaluate_model(y_true, y_pred, model_name='Model'):
    """모델 성능 평가"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{model_name} 성능:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    
    return {'rmse': rmse, 'mae': mae, 'r2': r2}

# 결과 저장
results = {}

### 5.1 Linear Regression

In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_val)
results['Linear Regression'] = evaluate_model(y_val, y_pred_lr, 'Linear Regression')

### 5.2 Ridge Regression

In [ ]:
# Ridge Regression
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_val)
results['Ridge'] = evaluate_model(y_val, y_pred_ridge, 'Ridge Regression')

### 5.3 Random Forest

In [ ]:
# Random Forest
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_val)
results['Random Forest'] = evaluate_model(y_val, y_pred_rf, 'Random Forest')

### 5.4 Gradient Boosting

In [ ]:
# Gradient Boosting
gb = GradientBoostingRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)
gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_val)
results['Gradient Boosting'] = evaluate_model(y_val, y_pred_gb, 'Gradient Boosting')

### 5.5 XGBoost

In [ ]:
# XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)

y_pred_xgb = xgb_model.predict(X_val)
results['XGBoost'] = evaluate_model(y_val, y_pred_xgb, 'XGBoost')

### 5.6 LightGBM

In [ ]:
# LightGBM
lgb_model = lgb.LGBMRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)]
)

y_pred_lgb = lgb_model.predict(X_val)
results['LightGBM'] = evaluate_model(y_val, y_pred_lgb, 'LightGBM')

---

## 6. 모델 평가 및 비교

In [ ]:
# 모델 성능 비교
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('rmse')

print('=' * 60)
print('모델 성능 비교 (RMSE 기준 정렬)')
print('=' * 60)
print(results_df)

In [ ]:
# 모델 성능 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE 비교
colors = ['green' if x == results_df['rmse'].min() else 'steelblue' for x in results_df['rmse']]
axes[0].barh(results_df.index, results_df['rmse'], color=colors)
axes[0].set_xlabel('RMSE')
axes[0].set_title('모델별 RMSE (낮을수록 좋음)')

# R² 비교
colors = ['green' if x == results_df['r2'].max() else 'steelblue' for x in results_df['r2']]
axes[1].barh(results_df.index, results_df['r2'], color=colors)
axes[1].set_xlabel('R² Score')
axes[1].set_title('모델별 R² (높을수록 좋음)')

plt.tight_layout()
plt.show()

### 6.1 Feature Importance

In [ ]:
# LightGBM Feature Importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][:15], feature_importance['importance'][:15])
plt.xlabel('Importance')
plt.title('LightGBM Feature Importance (상위 15개)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Feature Importance (상위 10개):")
print(feature_importance.head(10))

### 6.2 예측 vs 실제 비교

In [ ]:
# 예측 vs 실제 산점도 (최고 성능 모델)
best_model_name = results_df['rmse'].idxmin()
print(f"최고 성능 모델: {best_model_name}")

# 최고 성능 모델의 예측값 선택
if best_model_name == 'LightGBM':
    best_pred = y_pred_lgb
elif best_model_name == 'XGBoost':
    best_pred = y_pred_xgb
elif best_model_name == 'Random Forest':
    best_pred = y_pred_rf
elif best_model_name == 'Gradient Boosting':
    best_pred = y_pred_gb
else:
    best_pred = y_pred_lgb  # 기본값

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 산점도
axes[0].scatter(y_val, best_pred, alpha=0.5, s=20)
max_val = max(y_val.max(), max(best_pred))
axes[0].plot([0, max_val], [0, max_val], 'r--', label='Perfect Prediction')
axes[0].set_xlabel('실제 대여량')
axes[0].set_ylabel('예측 대여량')
axes[0].set_title(f'{best_model_name}: 실제 vs 예측')
axes[0].legend()

# 잔차 분포
residuals = y_val - best_pred
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='red', linestyle='--')
axes[1].set_xlabel('잔차 (실제 - 예측)')
axes[1].set_ylabel('빈도')
axes[1].set_title(f'{best_model_name}: 잔차 분포')

plt.tight_layout()
plt.show()

---

## 7. 최종 예측 및 제출 파일 생성

In [ ]:
# K-Fold Cross Validation으로 최종 모델 학습
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# OOF predictions
oof_lgb = np.zeros(len(X))
test_preds_lgb = np.zeros(len(X_test))

cv_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n===== Fold {fold + 1}/{n_folds} =====")
    
    X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]
    
    # LightGBM
    lgb_fold = lgb.LGBMRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    lgb_fold.fit(X_tr, y_tr, eval_set=[(X_vl, y_vl)])
    
    oof_lgb[val_idx] = lgb_fold.predict(X_vl)
    test_preds_lgb += lgb_fold.predict(X_test) / n_folds
    
    # Fold RMSE
    fold_rmse = np.sqrt(mean_squared_error(y_vl, oof_lgb[val_idx]))
    cv_scores.append(fold_rmse)
    print(f"Fold {fold + 1} RMSE: {fold_rmse:.4f}")

print(f"\n===== Cross Validation 결과 =====")
print(f"평균 RMSE: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

In [ ]:
# OOF 전체 성능
oof_rmse = np.sqrt(mean_squared_error(y, oof_lgb))
print(f"OOF RMSE: {oof_rmse:.4f}")

In [ ]:
# 음수 예측값 처리 (대여량은 0 이상)
test_preds_lgb = np.maximum(test_preds_lgb, 0)

print(f"예측값 통계:")
print(f"  최소: {test_preds_lgb.min():.2f}")
print(f"  최대: {test_preds_lgb.max():.2f}")
print(f"  평균: {test_preds_lgb.mean():.2f}")

In [ ]:
# 제출 파일 생성
submission_df = pd.DataFrame({
    'id': test_ids,
    'count': test_preds_lgb
})

# 저장
submission_df.to_csv('submission_seoul_bike.csv', index=False)

print("제출 파일 생성 완료!")
print(submission_df.head(10))

In [ ]:
# 예측값 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train vs Test 예측 분포
axes[0].hist(y, bins=50, alpha=0.5, label='Train (실제)', density=True)
axes[0].hist(test_preds_lgb, bins=50, alpha=0.5, label='Test (예측)', density=True)
axes[0].set_xlabel('대여량')
axes[0].set_ylabel('밀도')
axes[0].set_title('Train 실제 vs Test 예측 분포')
axes[0].legend()

# 시간대별 예측 평균
test_df_pred = test_df.copy()
test_df_pred['predicted_count'] = test_preds_lgb
hourly_pred = test_df_pred.groupby('hour')['predicted_count'].mean()

axes[1].bar(hourly_pred.index, hourly_pred.values, color='coral', edgecolor='black')
axes[1].set_xlabel('시간 (Hour)')
axes[1].set_ylabel('평균 예측 대여량')
axes[1].set_title('시간대별 평균 예측 대여량')
axes[1].set_xticks(range(24))

plt.tight_layout()
plt.show()

---

## 8. AutoGluon 모델링

**AutoGluon**은 Amazon에서 개발한 AutoML 라이브러리로, 자동으로 여러 모델을 학습하고 앙상블합니다.

### AutoGluon Presets

| Preset | 설명 | 학습 시간 | 성능 |
|--------|------|----------|------|
| `medium_quality` | 빠른 학습, 중간 성능 | 짧음 | 중간 |
| `best_quality` | 최고 성능, 스태킹 앙상블 | 김 | 최고 |

In [ ]:
# AutoGluon 라이브러리 로드
from autogluon.tabular import TabularDataset, TabularPredictor

print("AutoGluon 라이브러리 로드 완료!")

### 8.1 AutoGluon용 데이터 준비

AutoGluon은 자동 전처리를 수행하므로 원본 데이터를 사용합니다.

In [ ]:
# AutoGluon용 데이터 준비
# 특성 공학이 적용된 데이터 사용 (train_df, test_df)

# AutoGluon용 데이터프레임 생성
train_ag = train_df.copy()
test_ag = test_df.copy()

# id 컬럼 제거 (AutoGluon은 자동으로 처리)
train_ag = train_ag.drop(columns=['id'], errors='ignore')
test_ag_ids = test_df['id'].copy()
test_ag = test_ag.drop(columns=['id'], errors='ignore')

# 타겟 컬럼명
target_col = 'count'

print(f"AutoGluon Train shape: {train_ag.shape}")
print(f"AutoGluon Test shape: {test_ag.shape}")

### 8.2 AutoGluon - Medium Quality 모델

빠른 학습과 적당한 성능을 제공하는 프리셋입니다.

In [ ]:
# AutoGluon Medium Quality 모델 학습
predictor_medium = TabularPredictor(
    label=target_col,
    problem_type='regression',
    eval_metric='root_mean_squared_error',
    path='AutoGluon_SeoulBike_medium'
)

# 모델 학습 (tuning_data 없이 - medium_quality는 bagging 사용 안함)
predictor_medium.fit(
    train_data=train_ag,
    presets='medium_quality',
    time_limit=180,              # 3분
    verbosity=2
)

print('\n===== AutoGluon Medium Quality 학습 완료! =====')

In [ ]:
# Medium Quality 모델 리더보드 확인
print('===== AutoGluon Medium Quality - 모델 리더보드 =====')
leaderboard_medium = predictor_medium.leaderboard(silent=True)
print(leaderboard_medium)

In [ ]:
# Medium Quality 모델로 검증 데이터 예측 및 평가
# Train/Validation 분할 (이전과 동일한 분할 사용)
train_ag_split, val_ag_split = train_test_split(train_ag, test_size=0.2, random_state=42)

y_pred_ag_medium = predictor_medium.predict(val_ag_split.drop(columns=[target_col]))
y_true_ag = val_ag_split[target_col]

# 성능 평가
rmse_medium = np.sqrt(mean_squared_error(y_true_ag, y_pred_ag_medium))
mae_medium = mean_absolute_error(y_true_ag, y_pred_ag_medium)
r2_medium = r2_score(y_true_ag, y_pred_ag_medium)

print('\n===== AutoGluon Medium Quality 성능 =====')
print(f'  RMSE: {rmse_medium:.4f}')
print(f'  MAE:  {mae_medium:.4f}')
print(f'  R²:   {r2_medium:.4f}')

# 결과 저장
results['AutoGluon (Medium)'] = {'rmse': rmse_medium, 'mae': mae_medium, 'r2': r2_medium}

### 8.3 AutoGluon - Best Quality 모델

최고 성능을 위한 프리셋으로, 스태킹 앙상블과 K-Fold CV를 수행합니다.

**참고**: `best_quality` 프리셋은 bagging 모드를 사용하므로 `tuning_data` 없이 학습합니다.

In [ ]:
# AutoGluon Best Quality 모델 학습
predictor_best = TabularPredictor(
    label=target_col,
    problem_type='regression',
    eval_metric='root_mean_squared_error',
    path='AutoGluon_SeoulBike_best'
)

# 모델 학습 (best_quality는 tuning_data 없이!)
predictor_best.fit(
    train_data=train_ag,
    presets='best_quality',
    time_limit=300,              # 5분
    verbosity=2
)

print('\n===== AutoGluon Best Quality 학습 완료! =====')

In [ ]:
# Best Quality 모델 리더보드 확인
print('===== AutoGluon Best Quality - 모델 리더보드 =====')
leaderboard_best = predictor_best.leaderboard(silent=True)
print(leaderboard_best)

In [ ]:
# Best Quality 모델로 검증 데이터 예측 및 평가
y_pred_ag_best = predictor_best.predict(val_ag_split.drop(columns=[target_col]))

# 성능 평가
rmse_best = np.sqrt(mean_squared_error(y_true_ag, y_pred_ag_best))
mae_best = mean_absolute_error(y_true_ag, y_pred_ag_best)
r2_best = r2_score(y_true_ag, y_pred_ag_best)

print('\n===== AutoGluon Best Quality 성능 =====')
print(f'  RMSE: {rmse_best:.4f}')
print(f'  MAE:  {mae_best:.4f}')
print(f'  R²:   {r2_best:.4f}')

# 결과 저장
results['AutoGluon (Best)'] = {'rmse': rmse_best, 'mae': mae_best, 'r2': r2_best}

### 8.4 전체 모델 성능 비교 (기존 모델 + AutoGluon)

In [ ]:
# 전체 모델 성능 비교 (AutoGluon 포함)
results_df_all = pd.DataFrame(results).T
results_df_all = results_df_all.sort_values('rmse')

print('=' * 70)
print('전체 모델 성능 비교 (RMSE 기준 정렬) - AutoGluon 포함')
print('=' * 70)
print(results_df_all)

# 최고 성능 모델
best_model = results_df_all['rmse'].idxmin()
print(f"\n최고 성능 모델: {best_model}")
print(f"  RMSE: {results_df_all.loc[best_model, 'rmse']:.4f}")
print(f"  R²:   {results_df_all.loc[best_model, 'r2']:.4f}")

In [ ]:
# 전체 모델 성능 시각화 (AutoGluon 강조)
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# AutoGluon 모델 강조 색상
def get_color(model_name):
    if 'AutoGluon' in model_name:
        if 'Best' in model_name:
            return 'darkgreen'
        return 'lightgreen'
    return 'steelblue'

colors_rmse = [get_color(idx) for idx in results_df_all.index]

# RMSE 비교
axes[0].barh(results_df_all.index, results_df_all['rmse'], color=colors_rmse, edgecolor='black')
axes[0].set_xlabel('RMSE')
axes[0].set_title('전체 모델 RMSE 비교 (낮을수록 좋음)\n(녹색: AutoGluon)')
axes[0].axvline(x=results_df_all['rmse'].min(), color='red', linestyle='--', alpha=0.7)

# R² 비교
axes[1].barh(results_df_all.index, results_df_all['r2'], color=colors_rmse, edgecolor='black')
axes[1].set_xlabel('R² Score')
axes[1].set_title('전체 모델 R² 비교 (높을수록 좋음)\n(녹색: AutoGluon)')
axes[1].axvline(x=results_df_all['r2'].max(), color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# AutoGluon Best Quality Feature Importance
print('===== AutoGluon Best Quality - Feature Importance =====')
importance_ag = predictor_best.feature_importance(train_ag)
print(importance_ag.head(15))

### 8.5 AutoGluon 테스트 데이터 예측 및 제출 파일 생성

In [ ]:
# AutoGluon Best Quality 모델로 테스트 데이터 예측
test_preds_ag = predictor_best.predict(test_ag)

# 음수 값 처리 (대여량은 0 이상)
test_preds_ag = np.maximum(test_preds_ag, 0)

print(f"AutoGluon 예측 완료!")
print(f"예측값 통계:")
print(f"  최소: {test_preds_ag.min():.2f}")
print(f"  최대: {test_preds_ag.max():.2f}")
print(f"  평균: {test_preds_ag.mean():.2f}")

In [ ]:
# AutoGluon 제출 파일 생성
submission_ag = pd.DataFrame({
    'id': test_ag_ids,
    'count': test_preds_ag
})

# 저장
submission_ag.to_csv('submission_autogluon.csv', index=False)

print("AutoGluon 제출 파일 생성 완료!")
print(submission_ag.head(10))

---

## Summary

### 수행한 작업

1. **EDA (탐색적 데이터 분석)**
   - 타겟 변수(대여량) 분포 분석
   - 시간대별, 기상 조건별 대여량 패턴 분석
   - 상관관계 분석

2. **데이터 전처리**
   - 결측치 처리 (중앙값 대체)
   - 이상치 확인

3. **특성 공학**
   - 시간대 구분 (출근/퇴근/주간/야간)
   - 러시아워 플래그
   - 시간 사이클 특성 (sin/cos)
   - 불쾌지수
   - 미세먼지/초미세먼지 등급
   - 날씨 종합 점수

4. **모델링**
   - Linear Regression, Ridge
   - Random Forest, Gradient Boosting
   - XGBoost, LightGBM

5. **AutoGluon 모델링 (NEW)**
   - Medium Quality: 빠른 학습, 중간 성능
   - Best Quality: 스태킹 앙상블, 최고 성능

6. **모델 평가**
   - RMSE, MAE, R² 기반 비교
   - 기존 모델 vs AutoGluon 성능 비교
   - K-Fold Cross Validation

### 주요 인사이트

- **시간대**: 출퇴근 시간(8시, 18시)에 대여량이 가장 많음
- **기온**: 15-25°C의 쾌적한 온도에서 대여량 증가
- **강수**: 비가 오면 대여량이 크게 감소
- **미세먼지**: 미세먼지가 나쁘면 대여량 감소

### 생성된 파일
- `submission_seoul_bike.csv`: LightGBM K-Fold 예측 제출 파일
- `submission_autogluon.csv`: AutoGluon Best Quality 예측 제출 파일

---

## Summary

### 수행한 작업

1. **EDA (탐색적 데이터 분석)**
   - 타겟 변수(대여량) 분포 분석
   - 시간대별, 기상 조건별 대여량 패턴 분석
   - 상관관계 분석

2. **데이터 전처리**
   - 결측치 처리 (중앙값 대체)
   - 이상치 확인

3. **특성 공학**
   - 시간대 구분 (출근/퇴근/주간/야간)
   - 러시아워 플래그
   - 시간 사이클 특성 (sin/cos)
   - 불쾌지수
   - 미세먼지/초미세먼지 등급
   - 날씨 종합 점수

4. **모델링**
   - Linear Regression, Ridge
   - Random Forest, Gradient Boosting
   - XGBoost, LightGBM

5. **모델 평가**
   - RMSE, MAE, R² 기반 비교
   - K-Fold Cross Validation

### 주요 인사이트

- **시간대**: 출퇴근 시간(8시, 18시)에 대여량이 가장 많음
- **기온**: 15-25°C의 쾌적한 온도에서 대여량 증가
- **강수**: 비가 오면 대여량이 크게 감소
- **미세먼지**: 미세먼지가 나쁘면 대여량 감소

### 생성된 파일
- `submission_seoul_bike.csv`: 최종 예측 제출 파일